# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
import functions_notebook_arinew as fan
from datetime import timedelta
from scipy.stats import entropy
import os


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### Estrazione features, creazione dataframe, scrittura dataframes su disco
levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'
pf_capture_2 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'

# nome dataframe su disco
dfclean_disk = "df_clean.pkl"
dfpf1_disk = "dfpf1.pkl"
dfpf2_disk = "dfpf2.pkl"
dfpf3_disk = "dfpf3.pkl"

if not os.path.isfile(dfclean_disk):
	packets_cc = fn.load_packets_from_file(clean_capture)
	features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)
	df_cc = pd.DataFrame(features_cc)
	df_cc.to_pickle(dfclean_disk)
	features_cc.clear()

if not os.path.isfile(dfpf1_disk):
	pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_1)
	features_pf_15m_05h = fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)
	df = pd.DataFrame(features_pf_15m_05h)
	df_cc.to_pickle(dfpf1_disk)
	features_pf_15m_05h.clear()

if not os.path.isfile(dfpf2_disk):
	pkt_pf_15m_05h_2 = fn.load_packets_from_file(pf_capture_2)
	features_pf_15m_05h_2 = fn.extract_features_from_packets(pkt_pf_15m_05h_2, levels_of_interest)
	df1 = pd.DataFrame(features_pf_15m_05h_2)
	df_cc.to_pickle(dfpf2_disk)
	features_pf_15m_05h_2.clear()

if os.path.isfile(dfclean_disk):
	df_cc = pd.read_pickle(dfclean_disk)

if os.path.isfile(dfpf1_disk):
	df = pd.read_pickle(dfpf1_disk)

if os.path.isfile(dfpf3_disk):
	df1 = pd.read_pickle(dfpf3_disk)



In [4]:
fn.calcolo_features_binarie(df_cc)
fn.calcolo_features_binarie(df)
fn.calcolo_features_binarie(df1)

In [5]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'])
df['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])
df1['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])

In [6]:
df_cc.sort_values(by='frame.frame.time_utc')
df.sort_values(by='frame.frame.time_utc')
df1.sort_values(by='frame.frame.time_utc')

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,mbtcp,tcp,modbus,llc,icmpv6,stp,eth,data,icmp,ip
0,1,"May 21, 2018 13:12:37.348723000 ora legale Eur...",2018-05-21 11:12:37.348723+00:00,1526901157.348723000,0.000000000,0.000000000,0.000000000,0.000000000,1,60,...,0,0,0,0,0,0,1,0,0,0
1,1,"May 21, 2018 13:12:37.350527000 ora legale Eur...",2018-05-21 11:12:37.350527+00:00,1526901157.350527000,0.000000000,0.001804000,0.001804000,0.001804000,2,64,...,0,0,0,0,0,0,1,0,0,0
2,1,"May 21, 2018 13:12:37.350544000 ora legale Eur...",2018-05-21 11:12:37.350544+00:00,1526901157.350544000,0.000000000,0.000017000,0.000017000,0.001821000,3,60,...,0,1,0,0,0,0,1,0,0,1
3,1,"May 21, 2018 13:12:37.360801000 ora legale Eur...",2018-05-21 11:12:37.360801+00:00,1526901157.360801000,0.000000000,0.010257000,0.010257000,0.012078000,4,60,...,0,1,0,0,0,0,1,0,0,1
4,1,"May 21, 2018 13:12:37.360805000 ora legale Eur...",2018-05-21 11:12:37.360805+00:00,1526901157.360805000,0.000000000,0.000004000,0.000004000,0.012082000,5,60,...,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197773,1,"May 21, 2018 13:42:36.345657000 ora legale Eur...",2018-05-21 11:42:36.345657+00:00,1526902956.345657000,0.000000000,0.001220000,0.001220000,1798.996934000,197774,60,...,0,1,0,0,0,0,1,0,0,1
197774,1,"May 21, 2018 13:42:36.345673000 ora legale Eur...",2018-05-21 11:42:36.345673+00:00,1526902956.345673000,0.000000000,0.000016000,0.000016000,1798.996950000,197775,60,...,0,1,0,0,0,0,1,0,0,1
197775,1,"May 21, 2018 13:42:36.345854000 ora legale Eur...",2018-05-21 11:42:36.345854+00:00,1526902956.345854000,0.000000000,0.000181000,0.000181000,1798.997131000,197776,60,...,0,1,0,0,0,0,1,0,0,1
197776,1,"May 21, 2018 13:42:36.354401000 ora legale Eur...",2018-05-21 11:42:36.354401+00:00,1526902956.354401000,0.000000000,0.008547000,0.008547000,1799.005678000,197777,60,...,0,1,0,0,0,0,1,0,0,1


In [7]:
start_time = df['frame.frame.time_utc'].iloc[0]
end_time=start_time+timedelta(seconds=298.8)
end_time2 = end_time + timedelta(seconds=900)

In [8]:
start_time_2 = df1['frame.frame.time_utc'].iloc[0]
end_time_2=start_time_2+timedelta(seconds=299.16)
end_time2_2 = end_time_2 + timedelta(seconds=900)

In [9]:
df_cc1 = df[(df['frame.frame.time_utc'] >= start_time) & (df['frame.frame.time_utc'] < end_time)].copy()
df_attack = df[(df['frame.frame.time_utc'] >= end_time) & (df['frame.frame.time_utc'] <= end_time2)].copy()
df_cc2 = df[(df['frame.frame.time_utc'] > end_time2)].copy()

In [10]:
df1_cc1_2 = df1[(df1['frame.frame.time_utc'] >= start_time_2) & (df1['frame.frame.time_utc'] < end_time_2)].copy()
df1_attack_2 = df1[(df1['frame.frame.time_utc'] >= end_time_2) & (df1['frame.frame.time_utc'] <= end_time2_2)].copy()
df1_cc2_2 = df1[(df1['frame.frame.time_utc'] > end_time2_2)].copy()

In [11]:
df_cc_aggregation = fan.calcola_features(df_cc)
df_cc1_aggregation = fan.calcola_features(df_cc1)
df_cc2_aggregation = fan.calcola_features(df_cc2)
df_att_aggregation = fan.calcola_features(df_attack)

In [12]:
df1_cc1_aggregation = fan.calcola_features(df1_cc1_2)
df1_cc2_aggregation = fan.calcola_features(df1_attack_2)
df1_att_aggregation = fan.calcola_features(df1_cc2_2)

In [13]:
df1_att_aggregation.head()

,eth_count,ip_count,arp_count,udp_count,tcp_count,mbtcp_count,icmp_count,pkt_count,per_eth_count,per_ip_count,...,std_iplen,entropy_iplen,bytes_per_timeunit,pkt_per_timeunit,modbus_response_count,modbus_request_count,modb_req_resp_fraction,icmp_request_count,icmp_response_count,icmp_req_resp_fraction
0,550,541,7,0,25,1,516,550,1.0,0.983636,...,4.574718,0.311806,3095.2,110.0,NaN,NaN,NaN,516.0,0.0,0.0
1,376,368,5,0,44,2,324,376,1.0,0.978723,...,6.099469,0.575856,2177.6,75.2,NaN,NaN,NaN,324.0,0.0,0.0
2,61,55,4,0,50,2,5,61,1.0,0.901639,...,12.706953,1.392078,439.2,12.2,NaN,NaN,NaN,5.0,0.0,0.0
3,36,31,2,0,31,1,0,36,1.0,0.861111,...,14.491596,1.193777,253.6,7.2,NaN,NaN,NaN,NaN,0.0,0.0
4,56,50,4,0,50,2,0,56,1.0,0.892857,...,13.031032,1.207506,411.2,11.2,NaN,NaN,NaN,NaN,0.0,0.0


In [14]:
df1_cc1_aggregation.head()

,eth_count,ip_count,arp_count,udp_count,tcp_count,mbtcp_count,icmp_count,pkt_count,per_eth_count,per_ip_count,...,std_iplen,entropy_iplen,bytes_per_timeunit,pkt_per_timeunit,modbus_response_count,modbus_request_count,modb_req_resp_fraction,icmp_request_count,icmp_response_count,icmp_req_resp_fraction
0,28,25,2,0,25,1,0,28,1.0,0.892857,...,13.151136,1.207506,205.6,5.6,NaN,NaN,NaN,6.0,0.0,0.0
1,57,50,4,0,50,2,0,57,1.0,0.877193,...,13.799404,1.224276,411.2,11.4,NaN,NaN,NaN,NaN,0.0,0.0
2,50,44,4,0,44,2,0,50,1.0,0.880000,...,13.752091,1.228866,363.2,10.0,NaN,NaN,NaN,NaN,0.0,0.0
3,57,50,4,0,50,2,0,57,1.0,0.877193,...,13.799404,1.224276,411.2,11.4,NaN,NaN,NaN,NaN,0.0,0.0
4,35,31,2,0,31,1,0,35,1.0,0.885714,...,13.366080,1.168814,253.6,7.0,NaN,NaN,NaN,NaN,0.0,0.0


In [15]:
df1_cc2_aggregation.head()

,eth_count,ip_count,arp_count,udp_count,tcp_count,mbtcp_count,icmp_count,pkt_count,per_eth_count,per_ip_count,...,std_iplen,entropy_iplen,bytes_per_timeunit,pkt_per_timeunit,modbus_response_count,modbus_request_count,modb_req_resp_fraction,icmp_request_count,icmp_response_count,icmp_req_resp_fraction
0,210,203,5,0,25,1,178,210,1.0,0.966667,...,6.864732,0.622467,1202.4,42.0,NaN,NaN,NaN,178,0.0,0.0
1,741,731,7,0,51,2,680,741,1.0,0.986505,...,4.746099,0.387743,4227.2,148.2,NaN,NaN,NaN,680,0.0,0.0
2,1029,1025,2,0,31,1,994,1029,1.0,0.996113,...,2.852297,0.188184,5820.0,205.8,NaN,NaN,NaN,994,0.0,0.0
3,1055,1048,4,0,50,2,998,1055,1.0,0.993365,...,3.667312,0.276356,6000.0,211.0,NaN,NaN,NaN,998,0.0,0.0
4,1046,1040,4,0,44,2,996,1046,1.0,0.994264,...,3.467215,0.250730,5940.8,209.2,NaN,NaN,NaN,996,0.0,0.0
